In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from sklearn.metrics import confusion_matrix, accuracy_score
import skopt
import scipy
from skopt.space import Real
import category_encoders
from sklearn.naive_bayes import GaussianNB

PREDICCION_REAL = False
MAXIMIZAR_HIPERPARAMETROS = False
PARAMETROS = {
    
}



In [2]:
#APERTURA DE ARCHIVO DE ARCHIVOS
entrenamiento = pd.read_pickle("../Archivos/Arboles_entrenamiento.pkl")
test = pd.read_pickle("../Archivos/Arboles_validacion.pkl")

In [3]:
#FILTRADO DE COLUMNAS - NO REMOVER STAGE O FECHA
#fugas = ['ID','Opportunity_Name','Sales_Contract_No','Account_Name','Account_Owner','Opportunity_Owner','Last_Modified_By','Product_Family','Product_Name','ASP','ASP_(converted)']
#'Total_Taxable_Amount'
#columnas_fecha = ['Month','Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
#entrenamiento = entrenamiento.drop(columns=fugas)
#test = test.drop(columns=fugas)
#test = test.drop(columns=columnas_fecha)
#entrenamiento = entrenamiento.drop(columns=columnas_fecha)

In [4]:
#FECHAS A DIAS

columnas_fecha = ['Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
def fecha_a_dias(x):
    for columna in columnas_fecha:
        x[columna] = x[columna].apply(lambda x : (x - pd.to_datetime('01/01/2000', format='%m/%d/%Y')).days)

fecha_a_dias(entrenamiento)
fecha_a_dias(test)

In [5]:
objetivo = (entrenamiento['Stage'] == 1).astype(int)
entrenamiento = entrenamiento.drop(columns=['Stage'])
columnas_category = list(entrenamiento.select_dtypes(include=['category']).columns)
if 'Stage' in columnas_category : columnas_category.remove('Stage')

In [6]:
if not PREDICCION_REAL:
    test_label = (test['Stage'] == 1).astype(int)
    test = test.drop(columns=['Stage'])

In [7]:
def categoricas_a_numericas(train,test,label,usar_label):
    if (usar_label):
        columnas_object = list(train.select_dtypes(include=['category']).columns)
    else:
        columnas_object = list(test.select_dtypes(include=['category']).columns)
    if 'Stage' in columnas_object : columnas_object.remove('Stage')
    ohe = category_encoders.cat_boost.CatBoostEncoder(cols = columnas_object,return_df = True)
    ohe.fit(train,label)
    if (usar_label):
        columnas = ohe.transform(train,label)
        for columna in columnas_object:
            train[columna] = columnas[columna].copy()
    else:
        columnas = ohe.transform(test)
        for columna in columnas_object:
            test[columna] = columnas[columna].copy()
categoricas_a_numericas(entrenamiento,test,objetivo,False)
categoricas_a_numericas(entrenamiento,test,objetivo,True)

In [8]:
entrenamiento=entrenamiento.drop(columns = ["Year-Month"])
test = test.drop(columns = ["Year-Month"])

In [9]:
entrenamiento = entrenamiento.drop(columns = ["Opportunity_Duration_Ratio","Buro_Approved_by_Billing_Country_std","Quote_Expiry_Date","ASP","Opportunity_Taxable_Rate","ASP_by_Billing_Country_std","Total_Amount_by_Billing_Country_std","Opportunity_Duration_by_Product_Family_std","Total_Amount_by_Product_Family_std","Buro_Approved_by_Product_Family","Family_Duration","Territory_Duration"])
test = test.drop(columns = ["Opportunity_Duration_Ratio","Buro_Approved_by_Billing_Country_std","Quote_Expiry_Date","ASP","Opportunity_Taxable_Rate","ASP_by_Billing_Country_std","Total_Amount_by_Billing_Country_std","Opportunity_Duration_by_Product_Family_std","Total_Amount_by_Product_Family_std","Buro_Approved_by_Product_Family","Family_Duration","Territory_Duration"])

In [10]:
model = GaussianNB()

model.fit(entrenamiento,objetivo)
# make the prediction using the resulting model
preds = model.predict(test)
preds

array([1, 1, 1, ..., 0, 1, 1])

In [11]:
accuracy_score(test_label, preds)

0.6205980066445183

In [16]:
model.get_params()

{'priors': None, 'var_smoothing': 1e-09}

In [12]:
preds = model.predict(entrenamiento)
accuracy_score(objetivo, preds)

0.5930800430427945

In [13]:
if (PREDICCION_REAL):
    resultados['Opportunity_ID'] = test['Opportunity_ID']
    resultados['Target'] = pd.Series(preds)
    resultados = resultados.groupby('Opportunity_ID').mean()
    resultados = resultados.reset_index()
    #resultados['Target'] = resultados['Target'].apply(lambda x: int(x >= 0.5))    
    resultados.to_csv("prediccion.csv", index=False)
    resultados['Target'].value_counts()

In [14]:
if(not MAXIMIZAR_HIPERPARAMETROS): sys.exit()

SystemExit: 

C:\Users\gianb\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [64]:
from skopt import BayesSearchCV
from skopt.space import Real
from sklearn.model_selection import StratifiedKFold

# Classifier
bayes_cv_tuner = BayesSearchCV(
estimator = skl.naive_bayes.GaussianNB(),
search_spaces = {
        "var_smoothing" : Real(low=1e-20, high=1e-18, prior='uniform')
},
cv = skl.model_selection.TimeSeriesSplit(),
n_jobs = 30,
n_iter = 50,
verbose = 1,
refit = True,
random_state = 72
)

In [65]:
def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    

    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))

In [66]:
resultCAT = bayes_cv_tuner.fit(entrenamiento, objetivo, callback=status_print)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Model #1
Best ROC-AUC: 0.8599
Best params: OrderedDict([('var_smoothing', 1.64139377786995e-19)])

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   5 out of   5 | elapsed:    0.0s finished


Model #2
Best ROC-AUC: 0.8599
Best params: OrderedDict([('var_smoothing', 1.64139377786995e-19)])

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Model #3
Best ROC-AUC: 0.8599
Best params: OrderedDict([('var_smoothing', 1.64139377786995e-19)])

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   5 out of   5 | elapsed:    0.0s finished


Model #4
Best ROC-AUC: 0.8599
Best params: OrderedDict([('var_smoothing', 1.64139377786995e-19)])

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Model #5
Best ROC-AUC: 0.8599
Best params: OrderedDict([('var_smoothing', 1.64139377786995e-19)])

Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Model #49
Best ROC-AUC: 0.8603
Best params: OrderedDict([('var_smoothing', 8.822026760964564e-20)])



C:\Users\gianb\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   5 out of   5 | elapsed:    0.0s finished


Model #50
Best ROC-AUC: 0.8604
Best params: OrderedDict([('var_smoothing', 9.355906004013925e-20)])



In [61]:
print(bayes_cv_tuner.best_score_)
print(bayes_cv_tuner.best_params_)

0.8604073522106309
OrderedDict([('var_smoothing', 9.53548275748693e-20)])


In [62]:
preds = bayes_cv_tuner.predict(test.values)

In [63]:
accuracy_score(test_label, preds)

0.8586932447397564